處理空值有三種技巧
1.直接刪掉
2.填值(要填誰，會在特徵工程或是數據清理講到)
3.多設一個col紀錄誰是空的，用意是刪掉會導致模型不夠好，多創一個col紀錄是否空值，那被創造出來的col就可以當作特徵餵入模型

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

data = pd.read_csv('/Users/Rich/Desktop/Kaggle/Kaggle_team_learn/Internediate ML/melb_data.csv')



In [2]:
y = data.Price

melb_predictors = data.drop(['Price'], axis=1)
X = melb_predictors.select_dtypes(exclude=['object'])

X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                      random_state=0)

In [3]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Function for comparing different approaches
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=10, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

以上是設置

In [4]:
# 看有哪些col有缺失值[col for col in x_train.columns]是for loop的一行表達式
#any() 用于检查给定的可迭代对象（如列表、字符串、元组等）中是否存在至少一个元素为 True
cols_with_missing = [col for col in X_train.columns
                     if X_train[col].isnull().any()]
#提供另一種寫法dropna(axis=1) 會把有缺失值的col刪除
# Drop columns in training and validation data
reduced_X_train = X_train.dropna(axis = 1)
reduced_X_valid = X_valid.drop(cols_with_missing, axis=1)

print("MAE from Approach 1 (Drop columns with missing values):")
print(score_dataset(reduced_X_train, reduced_X_valid, y_train, y_valid))

MAE from Approach 1 (Drop columns with missing values):
183550.22137772635


In [5]:
from sklearn.impute import SimpleImputer

# Imputation
#SimpleImputer用於處理缺失值，常用的填補策略包括：平均值、中位數、最頻值、特定值等。
my_imputer = SimpleImputer()
#transform() 方法是用於將實例數據轉換為具有相同形狀的數據，通常是將原始數據按照訓練數據的規則進行轉換。它常用於數據標準化，正則化，維度縮減等預處理步驟。
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(my_imputer.transform(X_valid))

# Imputation removed column names; put them back
imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns

print("MAE from Approach 2 (Imputation):")
print(score_dataset(imputed_X_train, imputed_X_valid, y_train, y_valid))

MAE from Approach 2 (Imputation):
178166.46269899711


In [6]:
# Make copy to avoid changing original data (when imputing)
X_train_plus = X_train.copy()
X_valid_plus = X_valid.copy()

# Make new columns indicating what will be imputed
for col in cols_with_missing:
    X_train_plus[col + '_was_missing'] = X_train_plus[col].isnull()
    X_valid_plus[col + '_was_missing'] = X_valid_plus[col].isnull()

# Imputation
my_imputer = SimpleImputer()
imputed_X_train_plus = pd.DataFrame(my_imputer.fit_transform(X_train_plus))
imputed_X_valid_plus = pd.DataFrame(my_imputer.transform(X_valid_plus))

# Imputation removed column names; put them back
imputed_X_train_plus.columns = X_train_plus.columns
imputed_X_valid_plus.columns = X_valid_plus.columns

print("MAE from Approach 3 (An Extension to Imputation):")
print(score_dataset(imputed_X_train_plus, imputed_X_valid_plus, y_train, y_valid))

MAE from Approach 3 (An Extension to Imputation):
178927.503183954


In [7]:
X_train_plus.head()

,Rooms,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Propertycount,Car_was_missing,BuildingArea_was_missing,YearBuilt_was_missing
12167,1,5.0,3182.0,1.0,1.0,1.0,0.0,NaN,1940.0,-37.85984,144.9867,13240.0,False,True,False
6524,2,8.0,3016.0,2.0,2.0,1.0,193.0,NaN,NaN,-37.85800,144.9005,6380.0,False,True,True
8413,3,12.6,3020.0,3.0,1.0,1.0,555.0,NaN,NaN,-37.79880,144.8220,3755.0,False,True,True
2919,3,13.0,3046.0,3.0,1.0,1.0,265.0,NaN,1995.0,-37.70830,144.9158,8870.0,False,True,False
6043,3,13.3,3020.0,3.0,1.0,2.0,673.0,673.0,1970.0,-37.76230,144.8272,4217.0,False,False,False


In [8]:
# Shape of training data (num_rows, num_columns)
print(X_train.shape)

# Number of missing values in each column of training data
missing_val_count_by_column = (X_train.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

(10864, 12)
Car               49
BuildingArea    5156
YearBuilt       4307
dtype: int64


以下是習題有點難要跟上喔

In [13]:
X_full = pd.read_csv('/Users/Rich/Desktop/Kaggle/Kaggle_team_learn/Internediate ML/home-data-for-ml-course/train.csv', index_col='Id')
X_test_full = pd.read_csv('/Users/Rich/Desktop/Kaggle/Kaggle_team_learn/Internediate ML/home-data-for-ml-course/test.csv', index_col='Id')

In [14]:
#subset參數是指定要刪除的row，inplace=True是指在原數據上進行操作
X_full.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = X_full.SalePrice
X_full.drop(['SalePrice'], axis=1, inplace=True)

#下面這兩行是預設的，先不要管它
X = X_full.select_dtypes(exclude=['object'])
X_test = X_test_full.select_dtypes(exclude=['object'])

X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                      random_state=0)

In [15]:
X_train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
Id,,,,,,,,,,,,,,,,,,,,,
619,20,90.0,11694,9,5,2007,2007,452.0,48,0,...,774,0,108,0,0,260,0,0,7,2007
871,20,60.0,6600,5,5,1962,1962,0.0,0,0,...,308,0,0,0,0,0,0,0,8,2009
93,30,80.0,13360,5,7,1921,2006,0.0,713,0,...,432,0,0,44,0,0,0,0,8,2009
818,20,NaN,13265,8,5,2002,2002,148.0,1218,0,...,857,150,59,0,0,0,0,0,7,2008
303,20,118.0,13704,7,5,2001,2002,150.0,0,0,...,843,468,81,0,0,0,0,0,1,2006


In [16]:
#shape可顯示出數據的行列數
print(X_train.shape)

missing_val_count_by_column = (X_train.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

(1168, 36)
LotFrontage    212
MasVnrArea       6
GarageYrBlt     58
dtype: int64


In [ ]:
#總共有多少row
#num_rows = 

#有幾個col中有缺失值
#num_cols_with_missing =

#總共有幾個row有缺失值
#tot_missing =

In [16]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# 寫了一個定義分數的函數，主要是計算MAE
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

In [ ]:
# 找到有缺失值的col
#cols_with_missing = 

# 將有缺失值的col刪除
#reduced_X_train = 
#reduced_X_valid = 

In [18]:
#any() 用于检查给定的可迭代对象（如列表、字符串、元组等）中是否存在至少一个元素为 True
cols_with_missing = [col for col in X_train.columns
                    if X_train[col].isnull().any()] 
cols_with_missing

['LotFrontage', 'MasVnrArea', 'GarageYrBlt']

In [20]:
reduced_X_train = X_train.dropna(axis = 1)
reduced_X_train

,MSSubClass,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
Id,,,,,,,,,,,,,,,,,,,,,
619,20,11694,9,5,2007,2007,48,0,1774,1822,...,774,0,108,0,0,260,0,0,7,2007
871,20,6600,5,5,1962,1962,0,0,894,894,...,308,0,0,0,0,0,0,0,8,2009
93,30,13360,5,7,1921,2006,713,0,163,876,...,432,0,0,44,0,0,0,0,8,2009
818,20,13265,8,5,2002,2002,1218,0,350,1568,...,857,150,59,0,0,0,0,0,7,2008
303,20,13704,7,5,2001,2002,0,0,1541,1541,...,843,468,81,0,0,0,0,0,1,2006
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
764,60,9430,8,5,1999,1999,1163,0,89,1252,...,856,0,128,0,0,180,0,0,7,2009
836,20,9600,4,7,1950,1995,442,0,625,1067,...,436,290,0,0,0,0,0,0,2,2010
1217,90,8930,6,5,1978,1978,0,0,0,0,...,539,0,0,0,0,0,0,0,4,2010


In [22]:
reduced_X_valid = X_valid.drop(cols_with_missing, axis = 1)

In [23]:
print("MAE (Drop columns with missing values):")
print(score_dataset(reduced_X_train, reduced_X_valid, y_train, y_valid))

MAE (Drop columns with missing values):
17837.82570776256


In [ ]:
from sklearn.impute import SimpleImputer

# 用平均值填補缺失值使用SimpleImputer函數
#imputer = 
#imputed_X_train = 
#imputed_X_valid = 

# 使用SimpleImputer會把col名刪掉，所以要重新加上
#imputed_X_train.columns =
#imputed_X_valid.columns = 

In [ ]:
X_train

In [ ]:
imputed_X_train

In [ ]:
X_train.LotFrontage.describe()

In [ ]:
print("MAE (Imputation):")
print(score_dataset(imputed_X_train, imputed_X_valid, y_train, y_valid))

In [ ]:
# 這一步是要決定到底怎麼處理空值
#final_imputer = 
#final_X_train = 
#final_X_valid = 

In [ ]:
final_X_train

In [ ]:
final_X_train.columns = X_train.columns
final_X_valid.columns = X_valid.columns

In [ ]:
# Define and fit model
model = RandomForestRegressor(n_estimators=100, random_state=0)
model.fit(final_X_train, y_train)

# Get validation predictions and MAE
preds_valid = model.predict(final_X_valid)
print("MAE (Your approach):")
print(mean_absolute_error(y_valid, preds_valid))

In [ ]:
# X_test也要轉換
final_X_test = pd.DataFrame(final_imputer.transform(X_test))

# Fill in the line below: get test predictions
preds_test = model.predict(final_X_test)
